In [2]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [1]:
url="https://api.census.gov/data/2021/acs/acs1?get=NAME,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E&for=congressional%20district:*&in=state:*"
c=pd.read_csv(url)

NameError: name 'pd' is not defined